In [1]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd

In [2]:
from utils import get_all_top_30
from utils import get_keywords
from utils import get_search_news
from database import WRITEMEDataBase

In [3]:
db = WRITEMEDataBase(host='write-me.csuwdm9tp2la.ap-northeast-2.rds.amazonaws.com', password='readme2020')

In [4]:
now = datetime.now()
date = '%04d%02d%02d' % (now.year, now.month, now.day)
time = now.strftime("%p").lower()

# 1. News Crawling

In [5]:
df = get_all_top_30()
df.groupby('category').head(1)

,title,text,url,category,date
0,"177석 민주당, 국회의장 경선 대신 ‘추대’로 가나",6선 박병석·5선 김진표 최근 만나김 의원 출마 고심 “20일 최종 결...,https://news.naver.com/main/ranking/read.nhn?r...,정치,20200519
30,"채권단 ""두산베어스 팔아라""···창단 39년 만에 주인 바뀌나",두산중공업 채권단이 두산그룹과 프로야구 구단 두산베어스 매각을 ...,https://news.naver.com/main/ranking/read.nhn?r...,경제,20200519
60,[속보] 이태원 관련 서울 확진자 99명..구로 콜센터 뛰어넘어,[한국경제TV 김주리 기자]서울시는 19일 오후 6시 기준 이태원 클럽...,https://news.naver.com/main/ranking/read.nhn?r...,사회,20200519
90,"[속보]한국, WHO 집행이사국 확정…집행 이사 김강립 지명",[아시아경제 김수완 기자] 한국이 세계보건기구(WHO)의 집행 이...,https://news.naver.com/main/ranking/read.nhn?r...,세계,20200519
120,휴대폰 요금제 30년 만에 자율화…통신요금 오를까 내릴까,"통신사 '파격 요금제' 길 열리나국회 본회의 오르는 요금인가제통신업계""...",https://news.naver.com/main/ranking/read.nhn?r...,IT/과학,20200519


# 2. Get keywords

In [6]:
keywords = df.groupby('category').apply(get_keywords)

In [7]:
keywords

category
IT/과학     [노트, 출시, 텔레콤, 회장, 요금]
경제       [주류, 코로나, 두산, 부회장, 배달]
사회       [확진, 할머니, 등교, 병원, 간호사]
세계        [중국, 일본, 백신, 코로나, 한국]
정치       [청원, 의원, 국민, 성폭행, 청와대]
dtype: object

# 3. Get hints

In [8]:
cat = '정치'
key = keywords[cat][0]

cat_df = df[df['category']==cat]
key_df = cat_df[cat_df['text'].str.contains(key)]
n = len(key_df)
search_df = get_search_news(key, news=n).iloc[:n]
hints = get_keywords(pd.concat([key_df, search_df], axis=0), only_noun=True, n=6)
if key in hints:
    hints.remove(key)
hints = hints[:5]
print(cat, key, hints)
print(key_df['title'].iloc[0])

정치 청원 ['국민', '성폭행', '청와대', '개월', '사실']
[속보]"25개월 딸 성폭행한 초등생 처벌" 청원은 '허위'


# 4. To DB

In [14]:
db.select_keywords()

()

In [15]:
db.select_hints()

()

In [16]:
cat = '정치'
db.insert_keywords(keywords[cat], cat, date, time)

In [17]:
cat = '정치'
key = keywords[cat][0]
keywordID = db.keywordID(key, db.categoryID(cat))
db.insert_hints(keywordID, hints)

In [18]:
result = db.select_keywords()
result = pd.DataFrame(result)
result

,keywordID,categoryID,keywordName,date,time
0,1,1,청원,2020-05-19,pm
1,2,1,의원,2020-05-19,pm
2,3,1,국민,2020-05-19,pm
3,4,1,성폭행,2020-05-19,pm
4,5,1,청와대,2020-05-19,pm


In [19]:
result = db.select_hints()
result = pd.DataFrame(result)
result

,keywordID,hintID,hintName
0,1,1,수업
1,1,2,학생
2,1,3,학교
3,1,4,코로나
4,1,5,교육부


In [20]:
# db.delete_keywords()
# db.delete_hints()

In [ ]:
# db.close()